<div style="background-color:lightgray; color:black; padding:10px; border-radius:5px;">
  <h1>Sprint 4</h1>
</div>

## Tarea 4.1: Extracción de datos via APY y Web Scraping

## 🎯 **Objetivo:**
Extraer datos de una **API** y realizar **Web Scraping** utilizando **Python**.

---

### **1. Ejercicio Nº 1. Consumir una API** : Selecciona una API pública y extrae datos utilizando Python.

**"Recomendación Personalizada de Espacios de Trabajo Remoto en Barcelona Basada en Productividad y Condiciones Climáticas"**

**Objetivo**
El objetivo de esta primera fase es crear un dataset estructurado que combine información sobre los espacios de trabajo remoto disponibles en Barcelona y las condiciones climáticas actuales. A través de llamadas a las APIs públicas de Foursquare y OpenWeather, se recopilarán datos sobre cafeterías, coworkings y bibliotecas, junto con información climática en tiempo real (temperatura, humedad, viento, etc.). Este dataset servirá como base para futuras recomendaciones personalizadas que optimicen la productividad de los usuarios en función de sus preferencias y el clima.

**Justificación**
Con el auge del trabajo remoto, cada vez más personas buscan espacios fuera de casa que les proporcionen el ambiente adecuado para ser productivos. Sin embargo, factores como el clima y el tipo de lugar pueden influir considerablemente en la experiencia de trabajo. Por ejemplo, un día soleado puede hacer que los usuarios prefieran trabajar en una terraza, mientras que en días lluviosos opten por espacios cerrados y tranquilos.

Este proyecto, centrado en Barcelona, busca aprovechar las condiciones climáticas actuales para recomendar los mejores lugares para trabajar de manera remota. La creación de un dataset a partir de las APIs de Foursquare y OpenWeather es esencial para recopilar información relevante que pueda luego alimentar un sistema de recomendación inteligente, mejorando la experiencia del trabajador remoto al ofrecerle sugerencias personalizadas según el clima y sus preferencias.

In [1]:
#Librerias a utilizar
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import time

In [2]:
# Cargar las variables desde el archivo .env
load_dotenv(dotenv_path='/home/ngonzalez/mi_pagina_personal/.env')

# Acceder a las claves desde el entorno
foursquare_api_key = os.getenv('FOURSQUARE_API_KEY')
openweather_api_key = os.getenv('OPENWEATHER_API_KEY')

In [3]:
#Test API foursquare
url = "https://api.foursquare.com/v3/places/search?ll=41.3851,2.1734&limit=1"

headers = {
    "Accept": "application/json",
    "Authorization": f"{foursquare_api_key}"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Autenticación exitosa. Conexión establecida.")
    data = response.json()
    print(data)
else:
    print(f"Error en la solicitud: {response.status_code}")

Autenticación exitosa. Conexión establecida.
{'results': [{'fsq_id': '4af1b973f964a52083e221e3', 'categories': [{'id': 12082, 'name': 'Organization', 'short_name': 'Organization', 'plural_name': 'Organizations', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'VeryLikelyOpen', 'distance': 318, 'geocodes': {'drop_off': {'latitude': 41.387331, 'longitude': 2.175087}, 'main': {'latitude': 41.38759, 'longitude': 2.175354}, 'roof': {'latitude': 41.387574, 'longitude': 2.17532}}, 'link': '/v3/places/4af1b973f964a52083e221e3', 'location': {'address': 'Calle Palau de la Música 4-6', 'admin_region': 'Cataluña', 'country': 'ES', 'cross_street': 'Sant Pere més Alt', 'formatted_address': 'Calle Palau de la Música 4-6 (Sant Pere més Alt), 08003 Barcelona Catalunya', 'locality': 'Barcelona', 'postcode': '08003', 'region': 'Catalunya'}, 'name': 'Palau de la Música Catalana', 'related_places': {'children': [{'fsq_id': '

In [4]:
# TEST Api open weather - Coordenadas de Barcelona
latitud = 41.3851
longitud = 2.1734

# URL de la API de OpenWeather con tu clave API
openweather_api_key = openweather_api_key
url = f"http://api.openweathermap.org/data/2.5/weather?lat={latitud}&lon={longitud}&appid={openweather_api_key}&units=metric"

# Hacer la solicitud a OpenWeather
response = requests.get(url)

if response.status_code == 200:
    print("Autenticación exitosa. Conexión establecida.")
    data = response.json()
    print(data)
else:
    print(f"Error en la solicitud: {response.status_code}")

Autenticación exitosa. Conexión establecida.
{'coord': {'lon': 2.1692, 'lat': 41.387}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 23.23, 'feels_like': 22.99, 'temp_min': 21.83, 'temp_max': 24.7, 'pressure': 1023, 'humidity': 53, 'sea_level': 1023, 'grnd_level': 1017}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 120}, 'clouds': {'all': 20}, 'dt': 1729423242, 'sys': {'type': 2, 'id': 2003688, 'country': 'ES', 'sunrise': 1729404534, 'sunset': 1729443774}, 'timezone': 7200, 'id': 6544106, 'name': 'Ciutat Vella', 'cod': 200}


#### 1.1 Extracción de datos en Foursquare API

In [5]:
# Función para buscar lugares en Foursquare usando query y categorías
def buscar_lugares_por_query(api_key, coords, query, categorias, limite=50, total_resultados=200):
    """
    Función para buscar lugares en Foursquare usando query y categorías, con paginación para obtener más resultados.
    
    Parámetros:
    api_key (str): Clave API de Foursquare.
    coords (str): Coordenadas de la ciudad en formato "latitud,longitud".
    query (str): Palabra clave para buscar (e.g., 'wifi', 'terraza').
    categorias (str): Categorías de los lugares a buscar, separadas por comas.
    limite (int): Número máximo de lugares a devolver por solicitud (máximo permitido por Foursquare es 50).
    total_resultados (int): Número total de resultados que quieres obtener.
    
    Retorna:
    DataFrame con los lugares encontrados (nombre, dirección, categoría, coordenadas) y la query como tag.
    """
    places = []
    offset = 0  # Inicia en 0 para paginación
    while len(places) < total_resultados:
        # URL de la Foursquare API con paginación y query
        url = f"https://api.foursquare.com/v3/places/search?ll={coords}&query={query}&categories={categorias}&limit={limite}&offset={offset}"
        
        headers = {
            "Accept": "application/json",
            "Authorization": f"{api_key}"
        }
        
        # Realizar la solicitud a la API
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            for place in data['results']:
                categorias_lugar = [cat['name'] for cat in place['categories']]
                place_info = {
                    'nombre': place['name'],
                    'direccion': place['location']['formatted_address'],
                    'categorias': categorias_lugar,
                    'coordenadas': f"{place['geocodes']['main']['latitude']},{place['geocodes']['main']['longitude']}",
                    'tag': query  # Agregar la query como un tag
                }
                places.append(place_info)
            
            # Aumentar el offset para la siguiente página de resultados
            offset += limite
            
            # Agregar un retraso entre las solicitudes para evitar superar el límite de la API
            time.sleep(1)  # Puedes ajustar este valor según sea necesario
        
        else:
            print(f"Error en la solicitud: {response.status_code}")
            break
    
    # Convertir a DataFrame
    df_lugares = pd.DataFrame(places[:total_resultados])
    return df_lugares

In [6]:
# Coordenadas de Barcelona
barcelona_coords = "41.3851,2.1734"

# Categorías: Coworking Space (11128), Coffee Shop (13032), Library (11135)
categorias = "11128,13032"

# Lista de queries (palabras clave) para buscar
queries = ["coworking", "Outdoor seating"]

# DataFrame vacío para combinar resultados
df_combined = pd.DataFrame()

# Bucle para realizar una búsqueda por cada query y combinar los resultados
for query in queries:
    df_lugares = buscar_lugares_por_query(foursquare_api_key, barcelona_coords, query, categorias, limite=50, total_resultados=400)
    df_combined = pd.concat([df_combined, df_lugares], ignore_index=True)

# Eliminar duplicados basados en el nombre y dirección, si es necesario
df_combined.drop_duplicates(subset=['nombre', 'direccion'], inplace=True)

# Mostrar los resultados combinados
print(df_combined.head())

# Guardar el DataFrame combinado en un archivo CSV si es necesario
df_combined.to_csv('lugares_con_tags.csv', index=False)

               nombre                                          direccion  \
0           Coworking                  Calle Bruc, 42, Calders Catalunya   
1        IW Coworking  Carrer de Pau Claris, 90 (Gran Via de les Cort...   
2        MG Coworking                     Bruc, 168, Barcelona Catalunya   
3  Alparriuss Almazen    Consell de Cent, 523, 08013 Barcelona Catalunya   
4       Itnig Coffice    Pujades, 100 (Alaba), 08005 Barcelona Catalunya   

          categorias         coordenadas        tag  
0  [Coworking Space]  41.391672,2.173255  coworking  
1  [Coworking Space]  41.390296,2.170599  coworking  
2  [Coworking Space]  41.398883,2.163654  coworking  
3  [Coworking Space]  41.402324,2.180315  coworking  
4  [Coworking Space]   41.396518,2.19417  coworking  


#### 1.2 Extracción de datos en OpenWeather API

In [7]:
# Función para obtener datos climáticos usando la API de OpenWeather
def obtener_clima(openweather_api_key, lat, lon):
    """
    Obtiene datos climáticos actuales basados en latitud y longitud.
    
    Parámetros:
    openweather_api_key (str): Clave API de OpenWeather.
    lat (float): Latitud del lugar.
    lon (float): Longitud del lugar.
    
    Retorna:
    Un diccionario con los datos climáticos actuales.
    """
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={openweather_api_key}&units=metric"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()  # Retorna los datos climáticos como un JSON
    else:
        print(f"Error en la solicitud del clima: {response.status_code}")
        return None

# Ejemplo de uso con las coordenadas de un lugar en Barcelona
latitud = 41.3851  # Latitud de Barcelona
longitud = 2.1734  # Longitud de Barcelona

datos_climaticos = obtener_clima(openweather_api_key, latitud, longitud)

if datos_climaticos:
    # Extraer algunos datos importantes del clima
    temperatura = datos_climaticos['main']['temp']
    humedad = datos_climaticos['main']['humidity']
    descripcion_clima = datos_climaticos['weather'][0]['description']
    
    print(f"Temperatura: {temperatura}°C")
    print(f"Humedad: {humedad}%")
    print(f"Descripción del clima: {descripcion_clima}")

Temperatura: 23.23°C
Humedad: 52%
Descripción del clima: few clouds


#### 1.3 Integración de los datos climáticos en los lugares

In [8]:
# Función para agregar los datos climáticos a cada lugar
def agregar_clima_a_lugares(df_lugares, openweather_api_key):
    """
    Agrega datos climáticos a cada lugar en el DataFrame de lugares.
    
    Parámetros:
    df_lugares (DataFrame): DataFrame con los lugares de trabajo.
    openweather_api_key (str): Clave API de OpenWeather.
    
    Retorna:
    Un nuevo DataFrame con la información del clima agregada.
    """
    # Listas para almacenar los datos climáticos
    temperaturas = []
    humedades = []
    descripciones_clima = []
    
    # Iterar sobre cada lugar en el DataFrame
    for index, row in df_lugares.iterrows():
        latitud, longitud = row['coordenadas'].split(',')
        latitud, longitud = float(latitud), float(longitud)
        
        # Obtener datos climáticos
        clima = obtener_clima(openweather_api_key, latitud, longitud)
        
        if clima:
            temperaturas.append(clima['main']['temp'])
            humedades.append(clima['main']['humidity'])
            descripciones_clima.append(clima['weather'][0]['description'])
        else:
            # Si hay un error al obtener el clima, agregar valores nulos
            temperaturas.append(None)
            humedades.append(None)
            descripciones_clima.append(None)
    
    # Agregar las nuevas columnas al DataFrame de lugares
    df_lugares['temperatura'] = temperaturas
    df_lugares['humedad'] = humedades
    df_lugares['descripcion_clima'] = descripciones_clima
    
    return df_lugares

In [9]:
# Integrar el clima en el DataFrame de lugares
df_lugares_clima = agregar_clima_a_lugares(df_combined, openweather_api_key)

# Mostrar los primeros resultados con el clima incluido
print(df_lugares_clima.head())

# Guardar el DataFrame combinado con el clima en un archivo CSV
df_lugares_clima.to_csv('lugares_con_clima.csv', index=False)

               nombre                                          direccion  \
0           Coworking                  Calle Bruc, 42, Calders Catalunya   
1        IW Coworking  Carrer de Pau Claris, 90 (Gran Via de les Cort...   
2        MG Coworking                     Bruc, 168, Barcelona Catalunya   
3  Alparriuss Almazen    Consell de Cent, 523, 08013 Barcelona Catalunya   
4       Itnig Coffice    Pujades, 100 (Alaba), 08005 Barcelona Catalunya   

          categorias         coordenadas        tag  temperatura  humedad  \
0  [Coworking Space]  41.391672,2.173255  coworking        23.23       52   
1  [Coworking Space]  41.390296,2.170599  coworking        23.23       52   
2  [Coworking Space]  41.398883,2.163654  coworking        23.00       53   
3  [Coworking Space]  41.402324,2.180315  coworking        23.17       52   
4  [Coworking Space]   41.396518,2.19417  coworking        23.30       53   

  descripcion_clima  
0        few clouds  
1        few clouds  
2        few c

In [10]:
df_lugares_clima.head()

,nombre,direccion,categorias,coordenadas,tag,temperatura,humedad,descripcion_clima
0,Coworking,"Calle Bruc, 42, Calders Catalunya",[Coworking Space],"41.391672,2.173255",coworking,23.23,52,few clouds
1,IW Coworking,"Carrer de Pau Claris, 90 (Gran Via de les Cort...",[Coworking Space],"41.390296,2.170599",coworking,23.23,52,few clouds
2,MG Coworking,"Bruc, 168, Barcelona Catalunya",[Coworking Space],"41.398883,2.163654",coworking,23.00,53,few clouds
3,Alparriuss Almazen,"Consell de Cent, 523, 08013 Barcelona Catalunya",[Coworking Space],"41.402324,2.180315",coworking,23.17,52,few clouds
4,Itnig Coffice,"Pujades, 100 (Alaba), 08005 Barcelona Catalunya",[Coworking Space],"41.396518,2.19417",coworking,23.30,53,few clouds


### **Ejercicio 2. Obtener datos con Web Scraping**

El auge de los **K-Dramas** ha generado una gran comunidad de seguidores que buscan constantemente nuevas series que se ajusten a sus gustos. Sin embargo, con la creciente cantidad de contenido disponible, puede ser complicado elegir qué K-Drama ver a continuación. Por ello, este proyecto tiene como objetivo crear un DataFrame con información detallada de K-Dramas, incluyendo título, calificación, géneros, director, protagonistas y número de espectadores, con el propósito de desarrollar en el futuro una plataforma de recomendación personalizada de K-Dramas.

La idea es que, mediante el análisis de estos datos y el uso de la información proporcionada por los usuarios (como preferencias de géneros o actores favoritos), se pueda crear un sistema de recomendaciones que sugiere K-Dramas que encajen con los gustos individuales. Este proyecto sentará las bases para la creación de dicha plataforma, al extraer datos relevantes a través de técnicas de Web Scraping de sitios especializados, como MyDramaList.

**En esta primera fase, el scraping de K-Dramas permitirá generar un dataset estructurado con los siguientes campos:**

- Título del K-Drama
- Calificación (Rating)
- Géneros (Genres)
- Director
- Protagonistas (Main Cast)
- Número de espectadores (# of Watchers)

Este dataset no solo permitirá a los fanáticos de los K-Dramas explorar opciones, sino que también abrirá las puertas a la implementación de algoritmos de recomendación en función de los gustos y preferencias de los usuarios.

In [11]:
#Librerias a utilizar
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### 2.1: Extraer URLs de K-dramas de varias páginas

##### a. Primero se realizará la prueba para una pagina, luego se iterara. 

In [34]:
def obtener_lista_kdramas(pagina=1):
    url = f"https://mydramalist.com/shows/top?page={pagina}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    kdramas = []
    
    # Acceder a los K-dramas listados en la página
    for kdrama in soup.find_all("div", class_="box"):
        # Extraer el título
        titulo = kdrama.find("h6", class_="text-primary title").text.strip() if kdrama.find("h6", class_="text-primary title") else "No disponible"
        
        # Extraer la calificación
        calificacion = kdrama.find("span", class_="p-1-xs score").text.strip() if kdrama.find("span", class_="p-1-xs score") else "No disponible"
        
        # Extraer la URL del K-drama
        enlace = kdrama.find("a")
        url_kdrama = "https://mydramalist.com" + enlace["href"] if enlace and "href" in enlace.attrs else "No disponible"
        
        # Añadir los datos del K-drama a la lista
        kdramas.append({"Título": titulo, "Calificación": calificacion, "URL": url_kdrama})
    
    return kdramas

In [35]:
# Probar la función
lista_kdramas = obtener_lista_kdramas(pagina=1)

# Verificar si la lista fue extraída correctamente
if lista_kdramas is not None:
    # Mostrar los primeros registros para verificar
    print(lista_kdramas[:5])
else:
    print("No se pudo obtener la lista de K-dramas.")

[{'Título': 'Twinkling Watermelon', 'Calificación': 'No disponible', 'URL': 'https://mydramalist.com/739603-sparkling-watermelon'}, {'Título': 'Move to Heaven', 'Calificación': 'No disponible', 'URL': 'https://mydramalist.com/49231-move-to-heaven'}, {'Título': 'Hospital Playlist Season 2', 'Calificación': 'No disponible', 'URL': 'https://mydramalist.com/57173-hospital-playlist-2'}, {'Título': 'Weak Hero Class 1', 'Calificación': 'No disponible', 'URL': 'https://mydramalist.com/702267-weak-hero'}, {'Título': 'Nirvana in Fire', 'Calificación': 'No disponible', 'URL': 'https://mydramalist.com/9025-nirvana-in-fire'}]


##### b. Para 10 paginas

In [66]:
# Función para obtener la lista de URLs de K-dramas desde varias páginas
def obtener_urls_kdramas(num_paginas=10):
    base_url = "https://mydramalist.com/shows/top?page={}"
    urls_kdramas = []

    for pagina in range(1, num_paginas + 1):
        url = base_url.format(pagina)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extraer todas las URLs de los K-dramas de la página
        for kdrama in soup.find_all('h6', class_='text-primary title'):
            kdrama_url = "https://mydramalist.com" + kdrama.find('a')['href']
            urls_kdramas.append(kdrama_url)
    
    return urls_kdramas

In [67]:
# Probar la función para obtener las URLs de los K-dramas de 10 páginas
urls = obtener_urls_kdramas(num_paginas=10)
print(f"Se han extraído {len(urls)} URLs de K-dramas.")

Se han extraído 200 URLs de K-dramas.


In [68]:
print(urls)

['https://mydramalist.com/739603-sparkling-watermelon', 'https://mydramalist.com/49231-move-to-heaven', 'https://mydramalist.com/57173-hospital-playlist-2', 'https://mydramalist.com/702267-weak-hero', 'https://mydramalist.com/9025-nirvana-in-fire', 'https://mydramalist.com/52939-can-this-person-be-translated', 'https://mydramalist.com/54625-flower-of-evil', 'https://mydramalist.com/25560-moving', 'https://mydramalist.com/36269-doctor-playbook', 'https://mydramalist.com/750099-time-walking-on-memory', 'https://mydramalist.com/13544-reply-1988', 'https://mydramalist.com/697563-kai-duan', 'https://mydramalist.com/28674-joy-of-life', 'https://mydramalist.com/713331-the-lotus-casebook', 'https://mydramalist.com/2-1-litre-no-namida', 'https://mydramalist.com/25172-my-ajusshi', 'https://mydramalist.com/729705-hidden-love', 'https://mydramalist.com/28723-the-untamed', 'https://mydramalist.com/62295-luo-yao-knew-what-he-meant', 'https://mydramalist.com/54083-joy-of-life-season-2', 'https://mydr

#### 2.2. Extraer la información detallada de cada K-drama.

#### a.Prueba para un kdrama

In [64]:
# Función para extraer información de un K-Drama desde una página
def extraer_info_kdrama(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Título
    title_tag = soup.find('h1', class_='film-title')
    title = title_tag.text.strip() if title_tag else "No disponible"

    # Calificación (rating)
    rating_tag = soup.find('div', class_='hfs')
    rating = rating_tag.find('b').text if rating_tag else "No disponible"

    # Espectadores (Watchers)
    watcher = "No disponible"
    watchers_divs = soup.find_all('div', class_='hfs')
    for div in watchers_divs:
        if '# of Watchers:' in div.text:
            watcher = div.find('b').text.strip()
            break

    # Géneros
    genres_tag = soup.find('li', class_='show-genres')
    genres = ', '.join([genre.text.strip() for genre in genres_tag.find_all('a')]) if genres_tag else "No disponible"

    return {
        'Título': title,
        'Calificación': rating,
        'Géneros': genres,
        'Espectadores': watcher
    }

In [65]:
# Probar la función con una URL específica
detalles_kdrama = extraer_info_kdrama("https://mydramalist.com/739603-sparkling-watermelon")

# Mostrar los detalles obtenidos
print(detalles_kdrama)

{'Título': 'Twinkling Watermelon (2023)', 'Calificación': '9.2', 'Géneros': 'Romance, Youth, Drama, Fantasy', 'Espectadores': '86,749'}


##### b. Extraer informacion de varios Kdrama

In [69]:
# Crear un DataFrame con la información de los K-dramas
def crear_dataframe_kdramas(urls):
    kdramas_data = []
    
    for url in urls:
        info_kdrama = extraer_info_kdrama(url)
        kdramas_data.append(info_kdrama)
    
    df = pd.DataFrame(kdramas_data)
    return df

#### 2.3 Crear Dataframe

In [70]:
df_kdramas = crear_dataframe_kdramas(urls)

# Mostrar las primeras filas del DataFrame
print(df_kdramas.head())

                              Título Calificación  \
0        Twinkling Watermelon (2023)          9.2   
1              Move to Heaven (2021)          9.1   
2  Hospital Playlist Season 2 (2021)          9.1   
3           Weak Hero Class 1 (2022)          9.1   
4             Nirvana in Fire (2015)          9.1   

                                  Géneros Espectadores  
0          Romance, Youth, Drama, Fantasy       86,749  
1                             Life, Drama      105,409  
2           Romance, Life, Drama, Medical       71,742  
3                    Action, Youth, Drama       92,750  
4  Military, Historical, Drama, Political       38,806  


In [71]:
df_kdramas.head(20)

,Título,Calificación,Géneros,Espectadores
0,Twinkling Watermelon (2023),9.2,"Romance, Youth, Drama, Fantasy","86,749"
1,Move to Heaven (2021),9.1,"Life, Drama","105,409"
2,Hospital Playlist Season 2 (2021),9.1,"Romance, Life, Drama, Medical","71,742"
3,Weak Hero Class 1 (2022),9.1,"Action, Youth, Drama","92,750"
4,Nirvana in Fire (2015),9.1,"Military, Historical, Drama, Political","38,806"
5,Alchemy of Souls (2022),9.1,"Action, Historical, Romance, Fantasy","118,871"
6,Flower of Evil (2020),9.1,"Thriller, Romance, Crime, Melodrama","144,431"
7,Moving (2023),9.1,"Action, Thriller, Mystery, Supernatural","68,290"
8,Hospital Playlist (2020),9.1,"Friendship, Comedy, Life, Drama, Medical","113,356"
9,Lovely Runner (2024),9.0,"Music, Comedy, Romance, Fantasy","80,316"


In [72]:
# Guardar el DataFrame combinado con el clima en un archivo CSV
df_kdramas.to_csv('kdrama_database.csv', index=False)